In [1]:
import pandas as pd
import numpy as np
import ephem
import matplotlib.pyplot as plt
import os

%matplotlib inline

In [2]:
csvFiles = [f for f in os.listdir('texas_county')]
    

In [20]:
csvFiles[245]

'Swisher County.csv'

In [4]:
def EndtimeCivilTwilight(dates):
    
    '''
    End civil twilight function takes in the dates for all the stops in the 
    paper. 
    
    '''
    
    # set the location Houston
    end_twilight = []

    # set the location Houston
    obs = ephem.Observer()
    
    # converting the lat lon to radian which is the unit for ephem
    obs.lat = np.deg2rad(mean_lat)
    obs.long = np.deg2rad(mean_lon)

    obs.horizon = '-6'

    # all the civil twilight time for each unique time on which the stop occurred
    for d in all_dates_str:
        obs.date = d

        #foo = ephem.Date(houston.next_setting(ephem.Sun(), use_center=True))
        foo = obs.next_setting(ephem.Sun(), use_center=True)

        end_twilight.append(ephem.localtime(foo))

    return end_twilight

In [5]:
def beforeAfter(df, end_twilight):
    
    '''
    This function suppose to tell you 
    It takes input of the dataframe and a list of end civil twilight time for each date
    the output is a string for each stop determines whether stop occurs before or after 
    the sun has set. For before darkness fallen is marked BD and after darkness is marked
    with AD
    '''
    
# calculate the before dailight or after day light for each stops 
    bf_af = []

    for i in range(df.shape[0]):

        # get the timestamps for stops
        ts = df.iloc[i, 27]

        # check if the date will twilight 
        for d in range(len(end_twilight)):
            
            # find for each date
            if (ts.date() == end_twilight[d].date()):
        
                # compare the timestamp to the end civil twilight time and mark with in
                if (ts < end_twilight[d]):
                    bf_af.append('BD')

                else:      
                    bf_af.append('AD')
                    
    return bf_af


In [6]:
def ratioFactor(df):

    '''
    This function takes stops for each race and divide them by whether
    daylight is present. Then counts for each race in each county is tallied. 
    '''
    wb = df[df.driver_race == 'White'].groupby('light_or_dark').count().iloc[0, 0]
    wa = df[df.driver_race == 'White'].groupby('light_or_dark').count().iloc[1, 0]
    
    bb = df[df.driver_race == 'Black'].groupby('light_or_dark').count().iloc[0, 0]
    ba = df[df.driver_race == 'Black'].groupby('light_or_dark').count().iloc[1, 0]
    
    hb = df[df.driver_race == 'Hispanic'].groupby('light_or_dark').count().iloc[0, 0]
    ha = df[df.driver_race == 'Hispanic'].groupby('light_or_dark').count().iloc[1, 0]
   
    ab = df[df.driver_race == 'Asian'].groupby('light_or_dark').count().iloc[0, 0]
    aa = df[df.driver_race == 'Asian'].groupby('light_or_dark').count().iloc[1, 0]    
    # black
    br = ((bb / (ab + wb + bb + hb)), (ba / (aa + wa + ba + ha)))
    wr = ((wb / (ab + wb + bb + hb)), (wa / (aa + wa + ba + ha)))
    hr = ((hb / (ab + wb + bb + hb)), (ha / (aa + wa + ba + ha)))
    ar = ((ab / (ab + wb + bb + hb)), (aa / (aa + wa + ba + ha)))
    
    return br, wr, hr, ar

In [38]:
# iterating through the file by their county files
result = pd.DataFrame(columns=['County', 'White', 'Black', 'Hispanic', 'Asian'])

for f in csvFiles[29:]:
    
    df = pd.read_csv('texas_county' + '/' + f)
    
    #df = df[df['lat'].notnull() & df['lon'].notnull()]
    
    # calculating the average lat and lon
    mean_lat = df.lat.mean()
    mean_lon = df.lon.mean()
    
    # Find out the unqie dates for the 
    all_dates_str = df['stop_date'].unique()  + [' 00:00:00'] * len(df['stop_date'].unique())
    
    #create datetime objects for all the stop time
    df['stop_date_time'] = pd.to_datetime(df['stop_date'] + ' ' + df['stop_time'])
    
    # calculating the end of civil twilight for each date available 
    end_twilight = EndtimeCivilTwilight(all_dates_str)
    
    # Produce a 
    bf_af = beforeAfter(df, end_twilight)
    
    # attach the before or after 
    df['light_or_dark'] = pd.Series(bf_af)
    
    # Calculate the ratio of stop population before and after dark for each racial identity 
    br, wr, hr, ar= ratioFactor(df)
    
    result = result.append({'County': f.split('.')[0], 'Black': br, 'White': wr, 'Hispanic': hr, 'Asian': ar }, ignore_index=True)

/Users/tedu/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


IndexError: single positional indexer is out-of-bounds

In [39]:
result.shape

(16, 5)

In [40]:
result

,County,White,Black,Hispanic,Asian
0,Burnet County,"(0.812906810605, 0.809152208501)","(0.0309572948087, 0.0316690658383)","(0.148039371329, 0.149885092305)","(0.00809652325766, 0.00929363335606)"
1,Bastrop County,"(0.682812311891, 0.674268329684)","(0.0973153015771, 0.0965173394999)","(0.187407199326, 0.197561235619)","(0.0324651872065, 0.0316530951971)"
2,Comal County,"(0.67508605852, 0.666284263959)","(0.0550344234079, 0.0546192893401)","(0.257314974182, 0.265644670051)","(0.0125645438898, 0.0134517766497)"
3,Fayette County,"(0.628867297366, 0.624432459551)","(0.108220403989, 0.110134202935)","(0.206724622859, 0.212266399097)","(0.0561876757862, 0.0531669384172)"
4,Grimes County,"(0.647681491429, 0.640047393365)","(0.179319061537, 0.178823782852)","(0.143297258867, 0.149784575614)","(0.0297021881665, 0.0313442481689)"
5,Terry County,"(0.703575388027, 0.68126791972)","(0.0365853658537, 0.0402994584263)","(0.253048780488, 0.272602739726)","(0.00679046563193, 0.00582988212807)"
6,Concho County,"(0.668640897756, 0.675540869131)","(0.0561097256858, 0.0523723424154)","(0.264027431421, 0.263856848931)","(0.0112219451372, 0.00822993952241)"
7,Montague County,"(0.820910392926, 0.823772850604)","(0.0675010897316, 0.0671060925809)","(0.0998816862818, 0.0973360374928)","(0.0117068310605, 0.011785019322)"
8,Dallam County,"(0.767216009417, 0.768820391227)","(0.0303119482048, 0.0317624975301)","(0.191288993526, 0.18820391227)","(0.0111830488523, 0.0112131989725)"
9,Reagan County,"(0.676300578035, 0.69326591925)","(0.0332369942197, 0.0247673481351)","(0.287572254335, 0.279841723456)","(0.0028901734104, 0.00212500915952)"


In [ ]:
result['White']

In [ ]:
result.to_csv('result_1.csv') # 1 - 16

In [37]:
# rerun
result.to_csv('result_2.csv') # 18 - 28

In [41]:
# rerun
result.to_csv('result_12.csv') # 30 - 45

In [ ]:
result.to_csv('result_3.csv') # 47 - 52

In [ ]:
result.to_csv('result_4.csv') # 54 - 69

In [ ]:
result.to_csv('result_5.csv') # 71 - 83

In [ ]:
result.to_csv('result_6.csv') # 85 - 124

In [ ]:
result.to_csv('result_7.csv')# 126 - 143

In [11]:
result.to_csv('result_8.csv')# 145 - 208

In [33]:
# rerun
result.to_csv('result_9.csv')# 210 - 228

In [22]:
result.to_csv('result_10.csv')# 230 - 246

In [26]:
result.to_csv('result_11.csv')# 248 - 255

In [ ]:
df_1 = pd.read_csv('result_1.csv')
df_2 = pd.read_csv('result_2.csv')
df_3 = pd.read_csv('result_3.csv')

df_4 = pd.read_csv('result_4.csv')

df_5 = pd.read_csv('result_5.csv')



In [27]:
df_9 = pd.read_csv('result_9.csv')

df_10 = pd.read_csv('result_10.csv')

In [28]:
df_9.head()

,Unnamed: 0,County,White,Black,Hispanic,Asian
0,0,Aransas County,"(0.77298629293084864, 0.75937371240214258)","(0.028338210380409672, 0.026576019777503089)","(0.18158016325273371, 0.19703337453646477)","(0.017095333436008008, 0.017016893283889577)"
1,1,DeWitt County,"(0.70146935849958192, 0.68352701628923318)","(0.062358141201768011, 0.060960468812077867)","(0.23079679847091147, 0.24970202622169249)","(0.0053757018277386217, 0.0058104886769964241)"
2,2,Kinney County,"(0.52170880557977328, 0.506438111487297)","(0.030863121185701829, 0.029564661032763721)","(0.44219703574542285, 0.45823891658446858)","(0.0052310374891020054, 0.005758310895470662)"
3,3,Chambers County,"(0.4950190977598844, 0.49321782145825016)","(0.31622277278827293, 0.31092019080398098)","(0.16501496851450398, 0.17125246193408233)","(0.023743160937338702, 0.02460952580368652)"
4,4,Coryell County,"(0.72970680881996608, 0.7286109282422647)","(0.13496486551974801, 0.13187623436471363)","(0.1245456748243276, 0.12708360763660304)","(0.010782650835958323, 0.012429229756418697)"


In [29]:
df_10.head()

,Unnamed: 0,County,White,Black,Hispanic,Asian
0,0,Aransas County,"(0.77298629293084864, 0.75937371240214258)","(0.028338210380409672, 0.026576019777503089)","(0.18158016325273371, 0.19703337453646477)","(0.017095333436008008, 0.017016893283889577)"
1,1,DeWitt County,"(0.70146935849958192, 0.68352701628923318)","(0.062358141201768011, 0.060960468812077867)","(0.23079679847091147, 0.24970202622169249)","(0.0053757018277386217, 0.0058104886769964241)"
2,2,Kinney County,"(0.52170880557977328, 0.506438111487297)","(0.030863121185701829, 0.029564661032763721)","(0.44219703574542285, 0.45823891658446858)","(0.0052310374891020054, 0.005758310895470662)"
3,3,Chambers County,"(0.4950190977598844, 0.49321782145825016)","(0.31622277278827293, 0.31092019080398098)","(0.16501496851450398, 0.17125246193408233)","(0.023743160937338702, 0.02460952580368652)"
4,4,Coryell County,"(0.72970680881996608, 0.7286109282422647)","(0.13496486551974801, 0.13187623436471363)","(0.1245456748243276, 0.12708360763660304)","(0.010782650835958323, 0.012429229756418697)"


In [ ]:
print(df_1.shape)
df_1.head()

In [42]:
df_1 = pd.read_csv('result_1.csv')
df_2 = pd.read_csv('result_2.csv')
df_3 = pd.read_csv('result_3.csv')

df_4 = pd.read_csv('result_4.csv')

df_5 = pd.read_csv('result_5.csv')

df_6 = pd.read_csv('result_6.csv')
df_7 = pd.read_csv('result_7.csv')
df_8 = pd.read_csv('result_8.csv')

df_11 = pd.read_csv('result_11.csv')

df_12 = pd.read_csv('result_12.csv')
df_9 = pd.read_csv('result_9.csv')

df_10 = pd.read_csv('result_10.csv')

df = pd.concat([df_1, df_2, df_3, df_4, df_5, df_6, df_7, df_8, df_9, df_10, df_11, df_12], axis=0)

In [43]:
df.head()

,Unnamed: 0,County,White,Black,Hispanic,Asian
0,0,Callahan County,"(0.719591901801371, 0.70568872607576116)","(0.10329985652797705, 0.10647447638767221)","(0.15797863860991551, 0.1691752276286799)","(0.019129603060736491, 0.018661569907886703)"
1,1,Childress County,"(0.75872850442939033, 0.74888112919883931)","(0.07000173701580685, 0.065386317808488664)","(0.14903595622720167, 0.16020754438597354)","(0.022233802327601183, 0.025525008606698471)"
2,2,Fort Bend County,"(0.42432873093513301, 0.43148697700021937)","(0.21975935442593852, 0.21500262819159724)","(0.27137638536313541, 0.27035606822537056)","(0.084535529275793039, 0.083154326582812868)"
3,3,Brooks County,"(0.2426098090519401, 0.25081343888300195)","(0.017480376329120972, 0.017091566415258993)","(0.73478817569448307, 0.7266845353113508)","(0.0051216389244558257, 0.0054104593903883313)"
4,4,Tarrant County,"(0.64107690257230443, 0.65282726883997477)","(0.15127282420365187, 0.15251165726619292)","(0.179661468745835, 0.16943251002830323)","(0.027988804478208718, 0.025228563865529109)"


In [44]:
df.to_csv('final_result.csv')

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df_final.to_csv('all_result.csv')

In [ ]:
df = pd.read_csv('all_result.csv')

In [ ]:
df.head()

In [ ]:
plt.scatter(*zip(df.iloc[:, 1]))

In [ ]:
*list(df.iloc[:, 1])

In [ ]:
df.iloc[:, 1]

In [ ]:
type((df['Before'].iloc[0], df['After'].iloc[0]))

In [ ]:
#df['Before_After'] = pd.Series()
#df['Asian']
before_after = list()
for i in range(df.shape[0]):
    before_after.append(tuple((df['Before'].iloc[i], df['After'].iloc[i])))
    #df['Asian'].iloc[i] = (0, 0)
    

In [ ]:
df['Before_After'] = pd.Series(before_after)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df_ = pd.pivot_table(df, index=['County'], columns=['Race']).reset_index()

In [ ]:
df_.shape

In [ ]:
df_.head()

In [ ]:
df_['After']['Black'].iloc[0]

In [ ]:
df[]

In [ ]:
df_['After']['Black'].iloc[0]

In [ ]:
b_before_after = list()
w_before_after = list()
h_before_after = list()

for i in range(df_.shape[0]):
    b_before_after.append(tuple((df_['Before']['Black'].iloc[i], df_['After']['Black'].iloc[i])))
    w_before_after.append(tuple((df_['Before']['White'].iloc[i], df_['After']['White'].iloc[i])))
    h_before_after.append(tuple((df_['Before']['Hispanic'].iloc[i], df_['After']['Hispanic'].iloc[i])))

In [ ]:
df_['Black'] = b_before_after
df_['White'] = w_before_after
df_['Hipanic'] = h_before_after

In [ ]:
df_.head()

In [ ]:
df_['County']

In [ ]:
df_.iloc[:, -3:]

In [ ]:
a_before_after = list()

for i in range(df_.shape[0]):
    a_before_after.append((0, 0))

In [ ]:
df_['Asian'] = pd.Series(a_before_after)

In [ ]:
df_.iloc[:, -4:].shape

In [ ]:
county = df_['County']

In [ ]:
df_final['County'] = county
# df_final['County'] = df_['County']

In [ ]:
df_final.reset_index()

In [ ]:
df_final = df_.iloc[:, -4:]

df_final.merge(df_['County'])

df_final.head()

In [ ]:
df_final = pd.concat([df_final, df_['County']], axis=1)

In [ ]:
df_final.head()

In [ ]:
df_final.C

In [ ]:
df_final.shape

In [ ]:
df.head()

In [ ]:
df pd.pivot_table(df, index=['County', 'Race'])